# Search SDSS sample for observed 12CO 1-0 in ALMA archival data
## Search the ALMA archive using the `ALMAxmatch` module. Specifically we need the `runTargetQueryWithLines` method

In [2]:
#from astropy.table import Table
# read in sample
#MANGA_sample = Table.read('/Users/thbrown/samples/SDSS/dr15/manga/spectro/redux/v2_4_3/ryan-drpall-v2_4_3_unique_gals_only.csv')

The following are bash commands included during set up since terminal in the jupyter notebook server was unresponsive. Current solution is using the command line in a separate (but parallel) desktop session.

In [7]:
#! pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [8]:
#! pip install astroquery

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.7/832.7 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Generate SDSS superset

 - Must have spectroscopic redshift (z_spec < 0.5), 3" fibre aperture is ~18kpc at z = 0.5
 - Must have MPA/JHU stellar masses (log(mass) > 1)
 - Must have declination less than 47 (maximum ALMA declination)

"The adopted upper declination limit for ALMA is  + 47∘" (https://almascience.eso.org/proposing/proposers-guide) 

In [90]:
import pymysql
import numpy as np

#Query MySQL for mock survey statmorph results
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
#x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
#x = 'SELECT DISTINCT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u, dr15_ALMA_spaxels a WHERE u.objID = a.objID AND u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47 AND u.ra > 20 and u.ra < 35'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
data = np.array(db_data, dtype = float).T
table_id = np.array(db_data, dtype = str).T[0]

In [91]:
len(data[0])

8625

In [92]:
ra_sdss = data[1]
dec_sdss = data[2]

### Set path to custom `astroquery` module
This is a temporary hack to "use a version of ```astroquery``` that does not have a bug in the release date column. 

In [82]:
import sys
#sys.path = ['/Users/thbrown/astroquery'] + sys.path

### Import the `ALMAxmatch` tool
requires adding to the python path for now

In [83]:
! pwd

/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/notebooks


In [5]:
import sys
sys.path

['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/notebooks',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/arc/home/swilkinson/.local/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/arc/home/swilkinson/.ipython']

In [93]:
import sys
#sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/tools/ArchiveDev'] + sys.path
sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/'] + sys.path
from ALMAxmatch import archiveSearch

## Run the archive search
1. Create list of lists containing `SkyCoord` objects and search radii ie. [SkyCoord(ra,dec), searchRadius]*
2. Define the line rest frequencies (names are optional, output defaults to *line 0*, *line 1*, ...)
2. Pass tuple array along with lines of interest to `runTargetQueriesWithLines`, selecting only science data

### 1. `archiveSearch` takes either a tuple ('ra dec', 'radius') or lists of tuples.

In [94]:
from astropy import units as u
from astropy.coordinates import SkyCoord

# archive search impact parameter
searchradius = '20arcsec'
#searchradius = '3arcsec'
#searchradius = '100 arcsec'

# SkyCoord objects
targetCoords = [[SkyCoord(ra=ra*u.degree, dec=dec*u.degree), searchradius]
                   for ra, dec in zip(ra_sdss, dec_sdss)]

Generate target list in `mangaLineQuery`

In [95]:
# search using SkyCoord array
sdssALMAQuery = archiveSearch(targets=targetCoords)

display the target dictionary

In [96]:
display(sdssALMAQuery.targets)

{'coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (20.96776532, 13.29376443)>,
  <Angle 20. arcsec>],
 'coord=(20.92264503 deg 13.30939358 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (20.92264503, 13.30939358)>,
  <Angle 20. arcsec>],
 'coord=(20.68326218 deg 13.32744242 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (20.68326218, 13.32744242)>,
  <Angle 20. arcsec>],
 'coord=(20.41159143 deg 13.23469903 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (20.41159143, 13.23469903)>,
  <Angle 20. arcsec>],
 'coord=(20.49776739 deg 13.4193947 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (20.49776739, 13.4193947)>,
  <Angle 20. arcsec>],
 'coord=(21.06294102 deg 13.36423723 deg) radius=0.005555555555555556deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (21.06294102, 13.36423723)>,
  <Angle 20

### 2. Define lines of interest

Search for the following lines:

1. $^{12}$CO ($J=1-0$) @ 115.27120180 GHz rest frequency
2. $^{12}$CO ($J=2-1$) @ 230.53800000 GHz rest frequency
3. $^{12}$CO ($J=3-2$) @ 345.79598990 GHz rest frequency

In [97]:
# rest frame frequencies
rf_12CO_10 = 115.27120180 # 12CO J=1-0
#rf_12CO_21 = 230.53800000 # 12CO J=2-1
#rf_12CO_32 = 345.79598990 # 12CO J=3-2

rest_frequencies = [rf_12CO_10] # list of rest frequencies

line_names = ['12CO J=1-0'] # column names for observed boolean flags 

Run the query, pulling only science data where lines are observed

### 3. Search for ALMA data only using `runQueries`

In [98]:
sdssALMAQuery.runQueries(science=True)

ALMA archive querying: 100%|██████████| 8625/8625 [46:25<00:00,  3.10 target/s]  


coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg : No result
coord=(20.92264503 deg 13.30939358 deg) radius=0.005555555555555556deg : No result
coord=(20.68326218 deg 13.32744242 deg) radius=0.005555555555555556deg : No result
coord=(20.41159143 deg 13.23469903 deg) radius=0.005555555555555556deg : No result
coord=(20.49776739 deg 13.4193947 deg) radius=0.005555555555555556deg : No result
coord=(21.06294102 deg 13.36423723 deg) radius=0.005555555555555556deg : No result
coord=(20.86245252 deg 13.27854778 deg) radius=0.005555555555555556deg : No result
coord=(21.23006616 deg 13.32973321 deg) radius=0.005555555555555556deg : No result
coord=(22.21559614 deg 13.22077867 deg) radius=0.005555555555555556deg : No result
coord=(22.13119675 deg 13.29682548 deg) radius=0.005555555555555556deg : No result
coord=(21.39649449 deg 13.20878484 deg) radius=0.005555555555555556deg : No result
coord=(21.71776268 deg 13.2117126 deg) radius=0.005555555555555556deg : No result
coord=

In [99]:
display(sdssALMAQuery.queryResults)

{'coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg': <Table length=0>
 obs_publisher_did obs_collection facility_name ... pi_name collections pi_userid
                                                ...                              
       str33            str4           str3     ...  str256    str128     str256 
 ----------------- -------------- ------------- ... ------- ----------- ---------,
 'coord=(20.92264503 deg 13.30939358 deg) radius=0.005555555555555556deg': <Table length=0>
 obs_publisher_did obs_collection facility_name ... pi_name collections pi_userid
                                                ...                              
       str33            str4           str3     ...  str256    str128     str256 
 ----------------- -------------- ------------- ... ------- ----------- ---------,
 'coord=(20.68326218 deg 13.32744242 deg) radius=0.005555555555555556deg': <Table length=0>
 obs_publisher_did obs_collection facility_name ... pi_name collec

In [100]:
n=0 #counter
for targetname in sdssALMAQuery.targets.keys():
    if len(sdssALMAQuery.queryResults[targetname])>=1:
        n = n+1
        display(sdssALMAQuery.queryResults[targetname])

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [1,2]",Frequency resolution [1],sensitivity_10kms [1],Line sensitivity (native) [1],Polarizations [1]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2022.1.01515.S,ALMA,JAO,ALMA,uid://A001/X3577/X1d5.source.RGALX001p01MPAID057032.spw.13,cube,2,RGALX001p01MPAID057032,1.3641833333266502,0.8467088888888912,0.026434993956477505,Circle ICRS 1.364183 0.846709 0.013217,--,--,2048,1,2,11.648371216036596,59966.845086,59966.850353,272.16,272.16,0.002844537980577784,0.0028717754769107258,184928.80389175835,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X3577/X1d5,applicati,--,2022.1.01515.S,Proprietary,99.50764813696452,-59.917210770399734,3,961340708.673565,1000000000.0,J501:CM10 J502:CM02 J503:CM03 J504:CM12 J505:CM08 J506:CM05 N601:CM07 N602:CM01 N605:CM04 N606:CM06,F,2024-02-01T13:57:32.000,11.648371216036596,"[104.39..105.39GHz,564.45kHz,16.7mJy/beam@10km/s,988.3uJy/beam@native, XX YY]",104.89252098256128,1621.1236524055664,ALMA,,,,,--,"The rapid decline in baryon content towards the upper end of the galaxy mass function is one of the major open questions of galaxy evolution. It has been attributed to phenomena as diverse as environment, stellar mass-surface densities, bulge potential, and AGN feedback. We will use the ACA for a CO(1-0) survey of a purely mass-selected sample of 197 galaxies at z<0.3 with M_stellar 10^11-12 Msun, where the baryon deficit in galaxies is strongest. Our survey extends the xCOLD-GASS survey, which has been very successful in characterizing the cold gas in galaxies with stellar masses 10^9-10 Msun, but includes few galaxies above >10^11 Msun. Our sources fall within the SDSS Stripe 82, and have all SDSS spectroscopy, LOFAR and VLA radio imaging, and a rich sets of other photometric constraints. We will measure the local H2 mass function out to the highest masses, and obtain, e.g., integrated molecular gas masses, mass fractions, and gas consumption times as a function of the stellar, radio, and AGN properties of our sources, placing unbiased constraints on each of these characterstics in regulating the gas content and star formation efficiency in the most massive galaxies today.",RGALX001_a_03_7M,"Nesvadba, Nicole; Zovaro, Henry;",0.9882585229348659,5.186,uid://A001/X3577/X1d4,uid://A001/X3577/X1d5,uid://A002/X10275c0/X14104,An unbiased census of the molecular gas content in the most massive galaxies in the nearby Universe,S,TARGET,T,66.

### 3. Search using `runTargetQueriesWithLines`

In [101]:
# same thing as before?
sdssLineQuery = archiveSearch(targets=targetCoords)

In [ ]:
sdssLineQuery.runQueriesWithLines(restFreqs=rest_frequencies
                                        , line_names=line_names
                                        , science=True)

ALMA archive querying:  35%|███▍      | 2999/8625 [16:56<32:47,  2.86 target/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

ALMA archive querying:  83%|████████▎ | 7165/8625 [39:51<07:46,  3.13 target/s]

**Display the results**

In [103]:
n=0 #counter
for targetname in sdssLineQuery.targets.keys():
    if len(sdssLineQuery.queryResults[targetname])>=1:
        n = n+1
        display(sdssLineQuery.queryResults[targetname])

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [1,2]",Frequency resolution [1],sensitivity_10kms [1],Line sensitivity (native) [1],Polarizations [1]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2022.1.01515.S,ALMA,JAO,ALMA,uid://A001/X3577/X1d5.source.RGALX001p01MPAID057032.spw.13,cube,2,RGALX001p01MPAID057032,1.3641833333266502,0.8467088888888912,0.026434993956477505,Circle ICRS 1.364183 0.846709 0.013217,--,--,2048,1,2,11.648371216036596,59966.845086,59966.850353,272.16,272.16,0.002844537980577784,0.0028717754769107258,184928.80389175835,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X3577/X1d5,applicati,--,2022.1.01515.S,Proprietary,99.50764813696452,-59.917210770399734,3,961340708.673565,1000000000.0,J501:CM10 J502:CM02 J503:CM03 J504:CM12 J505:CM08 J506:CM05 N601:CM07 N602:CM01 N605:CM04 N606:CM06,F,2024-02-01T13:57:32.000,11.648371216036596,"[104.39..105.39GHz,564.45kHz,16.7mJy/beam@10km/s,988.3uJy/beam@native, XX YY]",104.89252098256128,1621.1236524055664,ALMA,,,,,--,"The rapid decline in baryon content towards the upper end of the galaxy mass function is one of the major open questions of galaxy evolution. It has been attributed to phenomena as diverse as environment, stellar mass-surface densities, bulge potential, and AGN feedback. We will use the ACA for a CO(1-0) survey of a purely mass-selected sample of 197 galaxies at z<0.3 with M_stellar 10^11-12 Msun, where the baryon deficit in galaxies is strongest. Our survey extends the xCOLD-GASS survey, which has been very successful in characterizing the cold gas in galaxies with stellar masses 10^9-10 Msun, but includes few galaxies above >10^11 Msun. Our sources fall within the SDSS Stripe 82, and have all SDSS spectroscopy, LOFAR and VLA radio imaging, and a rich sets of other photometric constraints. We will measure the local H2 mass function out to the highest masses, and obtain, e.g., integrated molecular gas masses, mass fractions, and gas consumption times as a function of the stellar, radio, and AGN properties of our sources, placing unbiased constraints on each of these characterstics in regulating the gas content and star formation efficiency in the most massive galaxies today.",RGALX001_a_03_7M,"Nesvadba, Nicole; Zovaro, Henry;",0.9882585229348659,5.186,uid://A001/X3577/X1d4,uid://A001/X3577/X1d5,uid://A002/X10275c0/X14104,An unbiased census of the molecular gas content in the most massive galaxies in the nearby Universe,S,TARGET,T,66.

In [104]:
print("galaxies with 12CO J=1-0:", n)

galaxies with 12CO J=1-0: 1


In [115]:
sdssLineQuery.queryResults['coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg']

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,pol_states,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,sensitivity_10kms,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256


In [ ]:
k=0 #counter
for targetname in herusLineQuery.targets.keys():
    if len(herusLineQuery.queryResults[targetname])>=1:
        # select only galaxies with all three lines
        if ((True in herusLineQuery.queryResults[targetname]['12CO J=1-0']) & 
            (True in herusLineQuery.queryResults[targetname]['13CO J=1-0']) &
            (True in herusLineQuery.queryResults[targetname]['C18O J=1-0'])):
            k = k+1
            display(herusLineQuery.queryResults[targetname])


In [ ]:
print("galaxies with 12CO, 13CO, AND C18O:", k)